# Imports

In [1]:
import src.acquire
import src.prepare

import pandas as pd
import numpy as np

# Bring in the Data

In [2]:
df = src.acquire.get_zillow_data()

CSV already generated, getting data from "data" directory


# Initial thoughts

• The columns with id values that were used for merging within mySQL (so not the actual id column, but things like airconditioningtypeid) can be dropped.

• Histograms to show distributions of data can be used to help impute missing values

• A threshold to drop rows and columns with a certain percentage of missing values will help remove irrelevant features

# Examining Nulls

In [3]:
nulls_by_column = src.prepare.nulls_by_col(df)
nulls_by_column.sort_values(by="percent_rows_missing", ascending=False, inplace=True)
# Examining the columns with the most missing values
nulls_by_column.head(30)

,number_missing_rows,percent_rows_missing
buildingclassdesc,77560,0.999807
buildingclasstypeid,77560,0.999807
finishedsquarefeet13,77533,0.999459
storytypeid,77525,0.999355
basementsqft,77525,0.999355
storydesc,77525,0.999355
yardbuildingsqft26,77505,0.999098
fireplaceflag,77403,0.997783
architecturalstyletypeid,77369,0.997345
architecturalstyledesc,77369,0.997345


In [4]:
# Consider the mid-range percent missing columns
nulls_by_column[20:40]

,number_missing_rows,percent_rows_missing
taxdelinquencyflag,74675,0.962617
finishedsquarefeet15,74548,0.960980
finishedsquarefeet50,71540,0.922204
finishedfloor1squarefeet,71540,0.922204
fireplacecnt,69288,0.893174
threequarterbathnbr,67470,0.869739
pooltypeid7,62497,0.805633
poolcnt,61402,0.791518
numberofstories,59979,0.773174
airconditioningdesc,52569,0.677654


In [5]:
null_rows = src.prepare.nulls_by_row(df)
null_rows.sort_values(by="pct_cols_missing", ascending=False, inplace=True)
null_rows

,num_cols_missing,pct_cols_missing,num_rows
25,48,70.58823529411765,3
24,47,69.11764705882352,3
23,46,67.64705882352942,5
22,45,66.17647058823529,50
21,44,64.70588235294117,79
20,43,63.23529411764706,29
19,42,61.76470588235294,26
18,41,60.29411764705882,29
17,40,58.82352941176471,230
16,39,57.35294117647059,285


# Goals
- Remove any properties that are likely to be something other than single unit properties.
- Create a function that will drop rows or columns based on the percent of values that are missing
- Decide how to handle the remaining missing values

## Getting only single unit residential properties

In [6]:
df = df[df.propertylandusetypeid.isin([260, 261, 262, 279])]

# Removing Columns with 50% missing values, and rows with 75% missing values 

In [7]:
shape_before = df.shape
df = src.prepare.handle_missing_values(df)
shape_after = df.shape
num_rows_dropped = shape_before[0] - shape_after[0]
num_columns_dropped = shape_before[1] - shape_after[1]
print(f'Number of rows dropped: {num_rows_dropped}, Number of columns dropped: {num_columns_dropped}')
print(f'Current shape: {df.shape}')

Number of rows dropped: 106, Number of columns dropped: 34
Current shape: (52374, 34)


# Dealing with Remaining Missing Values

## Bedroom/Bathroom Count

In [8]:
# Only keeping houses with bedroom and bathroom > 0
df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0)]
df.shape

(52292, 34)

## Unit Count

In [9]:
df.unitcnt.value_counts(dropna=False)

1.0    33782
NaN    18478
2.0       28
4.0        3
3.0        1
Name: unitcnt, dtype: int64

In [10]:
# 18478 missing values. We'll fill these in with the mode: 1
df.unitcnt = df.unitcnt.fillna(1.0)

In [11]:
# Filter to only one unit
df = df[df.unitcnt == 1.0]

In [12]:
df.shape

(52260, 34)

## Heaters

In [13]:
# b/c these properties are in southern california 
df.heatingorsystemdesc = df.heatingorsystemdesc.fillna("None")

# Dropping Columns

In [14]:
# Dropping 'id' Columns Used to Merge
df.drop(columns=['buildingqualitytypeid', 'heatingorsystemtypeid', 'propertylandusetypeid'], inplace=True)

In [15]:
# Handling duplicates
df.drop(columns='calculatedbathnbr', inplace=True)

In [16]:
# propertyzoningdesc = Description of the allowed land uses (zoning) for that property
# I'll drop this, b/c we're already filtering for single unit residential.
df.drop(columns='propertycountylandusecode', inplace=True)

In [17]:
# Dropping properyzoningdesc
df.drop(columns='propertyzoningdesc', inplace=True)

In [18]:
# Dropping census tract and block
df.drop(columns='censustractandblock', inplace=True)

## Filling Nulls with Medians

In [19]:
def fill_nulls_with_median(df):
    features = ['taxvaluedollarcnt',
                'calculatedfinishedsquarefeet',
                'taxamount',
                'fullbathcnt',
                'lotsizesquarefeet',
                'structuretaxvaluedollarcnt',
                'finishedsquarefeet12']
    
    for f in features:
        df[f] = df[f].fillna(df[f].median)
        
    return df

In [20]:
df = fill_nulls_with_median(df)

## Filling Nulls with Mode

In [21]:
def fill_nulls_with_mode(df):
    features = ['regionidzip',
                'regionidcity',
                'yearbuilt',
                'landtaxvaluedollarcnt']
    
    for f in features:
        df[f] = df[f].fillna(df[f].mode)
        
    return df

In [24]:
df = fill_nulls_with_mode(df)

In [25]:
df.isna().sum()

id                              0
parcelid                        0
bathroomcnt                     0
bedroomcnt                      0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
fullbathcnt                     0
latitude                        0
longitude                       0
lotsizesquarefeet               0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
unitcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
logerror                        0
transactiondate                 0
heatingorsystemdesc             0
propertylandusedesc             0
dtype: int64